## Programación Cuadrática
### Optimizacion de carteras con rotación restringida

Este cuaderno plantea el proceso de optimizar una cartera por segunda vez
imponiendo restricciones para que la rotación esté limitada

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import cvxpy as cp
import pickle

In [ ]:
with open('../data/stock_data.pkl', 'rb') as handle:
    stock_data = pickle.load(handle)

Para simplificar el ejercicio trabajaremos con un universo reducido
de 10 activos del IBEX.

In [ ]:
tickers = ['ACS','TEF','ITX','GRF','AMS','ENG','MAP','REP','AENA','VIS']

In [ ]:
close_dict = {tk: df.close for tk, df in stock_data.items() if tk in tickers}
stock_close = pd.DataFrame(close_dict)

____

**Funcion copiada del ejercicio 3_1**

In [ ]:
def efficient_frontier(returns, n_samples=50, gamma_low=-1, gamma_high=10):
    """
    construye un conjunto de problemas de programación cuádrática
    para inferir la frontera eficiente de Markovitz. 
    En cada problema el parámetro gamma se cambia para aumentar
    la penalización del riesgo en la función de maximización.
    """
    sigma = returns.cov().values
    mu = np.mean(returns, axis=0).values  
    n = sigma.shape[0]        
    w = cp.Variable(n)
    gamma = cp.Parameter(nonneg=True)
    ret = mu.T @ w
    risk = cp.quad_form(w, sigma)
    
    prob = cp.Problem(cp.Maximize(ret - gamma*risk), 
                      [cp.sum(w) == 1,  
                       w >= 0,
                       w <= 0.25]) 
    # Equivalente 
    #prob = cp.Problem(cp.Minimize(risk - gamma*ret), 
    #                  [cp.sum(w) == 1,  w >= 0])   
    risk_data = np.zeros(n_samples)
    ret_data = np.zeros(n_samples)
    gamma_vals = np.logspace(gamma_low, gamma_high, num=n_samples)
    
    portfolio_weights = []    
    for i in range(n_samples):
        gamma.value = gamma_vals[i]
        prob.solve()
        risk_data[i] = np.sqrt(risk.value)
        ret_data[i] = ret.value
        portfolio_weights.append(w.value)   
    return ret_data, risk_data, gamma_vals, portfolio_weights



In [ ]:
def get_optimal_portfolio(returns):
    ret_data, risk_data, gamma_vals, portfolio_weights = efficient_frontier(returns)
    sharpes = ret_data/risk_data 
    idx = np.argmax(sharpes)
    optimal_portfolio = pd.Series(portfolio_weights[idx],
                              index=returns.columns).round(3)
    return optimal_portfolio

____

#### Optimización Independiente
Primero haremos una optimización por separado para generar 2 carteras cada 6 meses

In [ ]:
data_close_h1 = stock_close.loc['2019-01-02':'2019-06-30'].dropna(axis=1)
data_close_h1

In [ ]:
returns_h1 = np.log(data_close_h1).diff().dropna()
returns_h1.head()

portfolio optimo para el primer semestre

In [ ]:
portfolio_h1 = get_optimal_portfolio(returns_h1)
portfolio_h1

ahora para el segundo semestre

In [ ]:
data_close_h2 = stock_close.loc['2019-07-01':'2019-12-31'].dropna(axis=1)
returns_h2 = np.log(data_close_h2).diff().dropna()
returns_h2.head()

In [ ]:
portfolio_h2 = get_optimal_portfolio(returns_h2)
portfolio_h2

___

Miramos el resultado semestral

In [ ]:
result_h2 = data_close_h2.iloc[-1]/data_close_h2.iloc[0]
result_h2

Calculamos propocionalmente, como si tuvieramos una cartera de 1€

In [ ]:
port_res = result_h2 * portfolio_h1
port_res

La cartera de H1 a final de año tendría los siguientes pesos

In [ ]:
port1_ath2 = port_res/port_res.sum()
port1_ath2

**Diferencia** respecto a la nueva asignación de la cartera h2

In [ ]:
portfolio_h2 - port1_ath2

**Rotación de la cartera** 

In [ ]:
rotacion = (portfolio_h2 - port1_ath2).abs().sum()
rotacion

____

#### Optimización con restricciones de rotación

In [ ]:
def efficient_frontier_max_rotation(returns, current_port,
                                    max_rotation=0.8,  
                                    n_samples=50, 
                                    gamma_low=-1, gamma_high=5):
    sigma = returns.cov().values
    mu = np.mean(returns, axis=0).values  
    n = sigma.shape[0]        
    
    w = cp.Variable(n)
         
    gamma = cp.Parameter(nonneg=True)
    ret = mu.T @ w
    risk = cp.quad_form(w, sigma)
    
    constraints = [
        cp.sum(w) == 1,  
        w >= 0,
        w <= 0.25,
        
        # restriccion para que la rotacion este limitada
        cp.sum(cp.abs(w - current_port.values)) <= max_rotation,
    ]
    
    prob = cp.Problem(cp.Maximize(ret - gamma*risk), constraints) 
    
    risk_data = np.zeros(n_samples)
    ret_data = np.zeros(n_samples)
    gamma_vals = np.logspace(gamma_low, gamma_high, num=n_samples)
    
    portfolio_weights = []
    buys_wg = []
    sells_wg = []
    for i in range(n_samples):
        gamma.value = gamma_vals[i]
        prob.solve(solver='ECOS')
        risk_data[i] = np.sqrt(risk.value)
        ret_data[i] = ret.value
        portfolio_weights.append(w.value)  
        
    return ret_data, risk_data, gamma_vals, portfolio_weights, buys_wg, sells_wg


In [ ]:
ret_data, risk_data, gamma_vals, portfolio_weights, port_buys, port_sells = efficient_frontier_max_rotation(
    returns_h2, port1_ath2, max_rotation=0.5
)

sharpes = ret_data/risk_data 
idx = np.argmax(sharpes)
portfolio_h2_rotation = pd.Series(portfolio_weights[idx],
                                  index=returns_h2.columns).round(3)
portfolio_h2_rotation

In [ ]:
portfolio_h2_rotation - port1_ath2

**Rotación limitada** 

In [ ]:
rotacion2 = (portfolio_h2_rotation - port1_ath2).abs().sum()
rotacion2

In [ ]:

res = pd.concat([port1_ath2, portfolio_h2_rotation, portfolio_h2], axis=1)
res.columns = ['current','portfolio_independiente','rotacion_limitada']
res